In [3]:
import pandas as pd
from citipy import citipy
import requests
import json
import numpy as np
import gmaps
from pprint import pprint
from config import gkey
gmaps.configure(api_key=gkey)

In [4]:
citi_data = pd.read_csv('../output/citi_data.csv')
fig = gmaps.figure(layout = {'width':'800px', 'height':'400px'}, zoom_level=1, center =(0,0))
heat = gmaps.heatmap_layer(citi_data[['latitude','longitude']], weights=citi_data['humidity'], max_intensity=100, point_radius=5)
fig.add_layer(heat)
fig


Figure(layout=FigureLayout(height='400px', width='800px'))

In [5]:
# create bool arrays for the ideal conditions
ideal_temp = (citi_data['temperature']>70) & (citi_data['temperature']<80)
ideal_windspeed = citi_data['windspeed']<10
ideal_cloudiness = citi_data['cloudiness']<25

# filter data from citi_data that meets the ideal conditions
vacation_spots = citi_data[ideal_temp & ideal_windspeed & ideal_cloudiness].copy(deep=True)
vacation_spots


,Unnamed: 0,name,country,latitude,longitude,temperature,cloudiness,humidity,windspeed
8,10,kumluca,TR,36.3703,30.2869,74.43,0.0,53.0,7.70
175,204,phuket,TH,8.0000,98.2500,79.02,20.0,94.0,2.13
242,278,bardiyah,LY,31.7561,25.0865,73.63,10.0,55.0,6.29
251,289,denia,ES,38.8408,0.1057,76.48,23.0,50.0,4.00
427,470,requena,PE,39.4883,-1.1004,70.77,3.0,38.0,7.47


In [8]:
# create a dataframe for capturing hotel info
hotels = pd.DataFrame(columns=['name', 'city', 'country', 'latitude','longitude'])
hotels['city'] = vacation_spots['name'].str.upper()
hotels['country']=vacation_spots['country']

# api call to gmaps to collect hotel info
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
parameters ={
    'key':gkey,
    'location':'',
    'keyword':'hotel',
    'radius':5000,
}

for index,place in vacation_spots.iterrows():
    parameters['location']=f'{place["latitude"]},{place["longitude"]}'
    response = requests.get(base_url,params=parameters).json()
    try:
        hotels.loc[index,'name']=response['results'][0]['name']
        hotels.loc[index, 'latitude'] = response['results'][0]['geometry']['location']['lat']
        hotels.loc[index, 'longitude'] = response['results'][0]['geometry']['location']['lng']
    except IndexError as e:
        print(e)

hotels.dropna(inplace=True)

# shotlisted hotels
hotels

list index out of range
list index out of range


,name,city,country,latitude,longitude
175,Trisara,PHUKET,TH,8.03923,98.2755
251,Hotel Los Angeles Denia,DENIA,ES,38.8588,0.0660602
427,PAGO DE THARSYS Bodega y Viñedos,REQUENA,PE,39.4986,-1.12188


In [9]:
# add hotel info to gmaps marker layer

hotels.dropna(subset=['name'], inplace=True)
info_box =[ {'name':row['name'],'city':row['city'],'country':row['country']} for index,row in hotels.iterrows()]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**info) for info in info_box]

markers = gmaps.marker_layer(hotels[['latitude','longitude']], info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='400px', width='800px'))